# 欢迎语
更新日志

## <i class="fa fa-repeat"></i> 切换pip软件源

In [ ]:
import os
import subprocess

def check_pip_list():
    output = !pip config list
    print(output)
    if "global.index-url='https://pypi.tuna.tsinghua.edu.cn/simple'" in output and "global.trusted-host='pypi.tuna.tsinghua.edu.cn'" in output:
        return True
    else:
        return False

if not check_pip_list():
    # 查找pip.conf文件
    pip_conf_file = os.popen("find / -name 'pip.conf'").read().strip()
    # 直接覆盖pip.conf文件
    with open(pip_conf_file, 'w') as f:
        f.write("[global]\nindex-url = https://pypi.tuna.tsinghua.edu.cn/simple\ntrusted-host = pypi.tuna.tsinghua.edu.cn")
        f.write("\nextra-index-url = https://pypi.org/simple")
    # 验证修改是否成功
    if check_pip_list():
        print("pip软件源切换成功！")
    else:
        print("pip软件源切换失败，请检查脚本是否正确运行。")
else:
    print("pip软件源本来就是清华源，无需切换")


In [ ]:
new_channels = [
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/msys2/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/conda-forge/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/peterjc123/",
    "https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud/pytorch/"
]

current_channels = !conda config --show channels
current_channels = ('').join(current_channels).split()

# print(current_channels)

for channel in new_channels:
    if channel in current_channels:
        print(f"此channel已经在列表中：{channel}")
    else:
        !conda config --add channels $channel
        
!conda config --set show_channel_urls yes

##  <i class="fa fa-spinner"></i> 装载自定义模块
把当前repo根目录加入 Python 搜索模块的路径列表，并求得根目录

In [ ]:
import os,sys

def find_repo_root():
    # 获取当前脚本的绝对路径
    try:
        # 如果当前运行环境是 Jupyter Notebook，使用当前工作目录
        script_path = os.path.abspath('.')
    except NameError:
        # 否则，使用 __file__ 变量
        script_path = os.path.abspath(__file__)
    # 判断当前脚本目录是否存在`.git`文件夹
    if os.path.isdir(os.path.join(script_path, '.git')):
        return script_path
    # 初始化存储结果的变量
    result = None
    # 循环求当前路径的父目录，直到找到`.git`文件夹
    while True:
        # 将当前路径的父目录赋值给当前路径
        script_path = os.path.dirname(script_path)
        # 判断当前路径是否存在`.git`文件夹
        if os.path.isdir(os.path.join(script_path, '.git')):
            # 如果存在，将当前路径存储在结果变量中
            result = script_path
        # 判断当前路径是否为根目录（即是否已经搜索到最外层）
        if script_path == '/':
            # 如果是，返回结果变量的值
            return result
        
repo_root_dir=find_repo_root()
sys.path.append(repo_root_dir)
# print(repo_root_dir)
# 获取repo所在的根目录
root_dir = os.path.dirname(repo_root_dir)
# 获取当前环境名称与当前环境content路径
from func.env import detect_environment
env_cb = detect_environment()
env_name = env_cb['env_name']
content_path = env_cb['content_path']

# 装载nbfunc中的hub涉及的所有ipynb函数
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

initGit()

## <i class="fa fa-arrow-down"></i> 安装必要依赖

In [ ]:
!pip uninstall transformers -y

In [ ]:
import subprocess

libraries_to_install = [

]

tools_to_install = [
    "iputils-ping",
]

def extract_library_name(library):
    """从库名中提取出包的名称"""
    return library.split("==")[0]

def get_installed_libraries():
    """获取已安装的库列表"""
    output = subprocess.check_output(["pip", "freeze"])
    installed_libraries = output.decode().split("\n")
    return set([extract_library_name(library) for library in installed_libraries])

def install_library(library):
    """安装指定的库"""
    !pip install $library --user

# 获取已安装的库列表
installed_libraries = get_installed_libraries()

# 找出未安装的库并安装
libraries_to_install_filtered = [library for library in libraries_to_install if extract_library_name(library) not in installed_libraries]
if libraries_to_install_filtered:
    print(f"以下库将被安装：{libraries_to_install_filtered}")
    for library in libraries_to_install_filtered:
        install_library(library)
else:
    print("所有需要安装的库都已经安装。")

# 尝试安装所有未安装的工具
update_needed = False
for tool in tools_to_install:
    exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
    if exit_code != 0:
        update_needed = True
        break
    else:
        print(f"【 {tool} 】已经安装，跳过安装")
if update_needed:
    !apt-get update
    for tool in tools_to_install:
        exit_code = subprocess.run(["dpkg", "-s", tool], capture_output=True).returncode
        if exit_code != 0:
            !apt-get install -y $tool

# 基础配置

In [ ]:
#需要安装ipywidgets，此镜像已经安装了对应的依赖包
# !conda install -c conda-forge jupyterlab 
# !conda install -c conda-forge ipywidgets
# !conda install -c conda-forge ipython
# !conda install -c conda-forge ipympl

# 实现显示/隐藏代码块的步骤：
# 安装jupyter_contrib_nbextentions
# （Ref: https://jupyter-contrib-nbextensions.readthedocs.io/en/latest/install.html#install-the-python-package）
# conda安装命令：conda install -c conda-forge jupyter_contrib_nbextensions
# python安装命令：pip install jupyter_contrib_nbextensions

# 安装jupyter_nbextentions_contributor，以便在Jupyter Notebook页面设置nbextentions
# （Ref: https://github.com/Jupyter-contrib/jupyter_nbextensions_configurator）
# conda安装命令：conda install -c conda-forge jupyter_nbextensions_configurator
# python安装命令：pip install jupyter_nbextensions_configurator

# !apt-get update && apt install net-tools

## <i class="fa fa-plane-departure"></i> 学术加速
首先要进行学术加速，这有利于拉取资源，详情请看：https://www.autodl.com/docs/network_turbo/  

In [ ]:
# from func.env import setProxy
# try:
#     proxy,region
# except NameError:
#     sp = setProxy()
#     proxy = sp['proxy']
#     region = sp['region']
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb
selProxy(True,'自供')
print(proxy)
print(proxyURL)

## 设置Git使用代理访问GitHub

In [ ]:
def set_git_proxy(proxyURL):
    # 设置Git在使用HTTP协议访问GitHub时使用代理服务器
    !git config --global http.https://github.com.proxy $proxyURL

    # 设置Git在使用HTTPS协议访问GitHub时使用代理服务器
    !git config --global https.https://github.com.proxy $proxyURL

    # 验证是否设置成功
    result = !git config --list
    # print(result)
    result = [str(item) for item in result] # 将结果转换为字符串列表
    http_proxy = f'http.https://github.com.proxy={proxyURL}'
    https_proxy = f'https.https://github.com.proxy={proxyURL}'
    if http_proxy in result and https_proxy in result:
        print('Git代理已成功设置。')
    else:
        print('Git代理设置失败。')

set_git_proxy(proxyURL)

## <i class="fa fa-star"></i> Notebook环境切换
在JupyterLab的Notebook中使用新的Conda环境

In [ ]:
from subprocess import getoutput
from IPython.display import clear_output
import ipywidgets as widgets

python_version = '3.10.6'

if env_name == 'AutoDL':
    custom_env_name = 'py' + python_version
elif env_name == 'OpenBayes':
    custom_env_name = 'base'

s = getoutput('conda info --env')
print(s)
if custom_env_name not in s:
    print(f'当前系统中没有{custom_env_name}环境，正在安装')
    # 创建Conda新的虚拟环境
    # 构建一个虚拟环境
    # 更新bashrc中的环境变量
    # 在 /openbayes/home 下创建一个新的环境：https://openbayes.com/docs/gear/conda/#%E4%BD%BF%E7%94%A8-conda-%E5%88%9B%E5%BB%BA%E6%96%B0%E7%9A%84%E7%8E%AF%E5%A2%83
    python_install_args = [
        'source /usr/local/etc/profile.d/conda.sh','&&',
        f'cd {root_dir}','&&',
        f'conda create -p {custom_env_name} python={python_version} -y','&&',
        'conda init bash','&&',
        'source /root/.bashrc -y'
    ]
    
    # !source /usr/local/etc/profile.d/conda.sh &&\
    # conda create -n $custom_env_name python=$python_version -y &&\
    # conda init bash && source /root/.bashrc -y
    
    python_install_cmd = (" ").join(python_install_args)
    !$python_install_cmd
    
else:
    print(f'已经安装了{custom_env_name}环境')

# clear_output(wait=True)
# print("Done!")

widgets.Button(
    value=False,
    description='环境部署成功!',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

## 交代Base_args

In [ ]:
from func.env import setProxy,getWebUIDir
import subprocess

def find_conda_env(name):
    # Get a list of all conda environments on the system
    output = subprocess.run(["conda", "env", "list"], capture_output=True)
    envs = output.stdout.decode().split("\n")

    # Find the path of the environment
    for env in envs:
        if name in env:
            path = env.split()[-1]
            print(f"The path of the environment '{name}' is {path}")
            return path
    else:
        print(f"Environment '{name}' not found.")
        return None



try:
    proxy,region
except NameError:
    cb=setProxy()
    proxy=cb['proxy']
    region=cb['region']
    clear_output(wait=True)

webUIDir=getWebUIDir()
print(webUIDir)

custom_env_dir = find_conda_env(custom_env_name)

base_args = [
    'source activate','&&',
    f'conda activate {custom_env_dir}','&&',
    proxy,'&&'
       ]

### 从源代码的方式安装triton

In [ ]:
# triton_repo_dir = os.path.join(root_dir,'triton')

# if not os.path.exists(triton_repo_dir):
#     !cd $root_dir &&\
#     git clone https://github.com/openai/triton.git

# !cd $root_dir &&\
# cd triton/python &&\
# pip install cmake &&\
# pip -v install -e .

### 通过PIP安装triton的最新nightly release

In [ ]:
triton_install_args = base_args[:]
triton_install_args += [
    'python -m pip install --user -U --pre triton'
]
triton_install_cmd = (" ").join(triton_install_args)
print(triton_install_cmd)
!$triton_install_cmd

## <i class="fa fa-bolt"></i> 安装Xformers和triton

大幅减少计算时间和显存占用，代价是生成的图每个都有点变化，强烈推荐安装!  
但不同核心代号的显卡需要不同的编译包，不能混用  
当然此笔记会帮你自动检测这台机器是否适合安装，而且会自动帮你安装合适的编译包

### 从源代码的方式安装Xformers

In [ ]:
import subprocess,os,time
'''
执行!python -m xformers.info命令会返回类似以下格式的信息，请参考

xFormers 0.0.16+bc08bbc.d20230122
memory_efficient_attention.cutlassF:               available
memory_efficient_attention.cutlassB:               available
memory_efficient_attention.flshattF:               available
memory_efficient_attention.flshattB:               available
memory_efficient_attention.smallkF:                available
memory_efficient_attention.smallkB:                available
memory_efficient_attention.tritonflashattF:        available
memory_efficient_attention.tritonflashattB:        available
swiglu.fused.p.cpp:                                available
is_triton_available:                               True
is_functorch_available:                            False
pytorch.version:                                   1.12.1+cu113
pytorch.cuda:                                      available
gpu.compute_capability:                            8.6
gpu.name:                                          NVIDIA GeForce RTX 3090
build.info:                                        available
build.cuda_version:                                1103
build.python_version:                              3.8.13
build.torch_version:                               1.12.1+cu113
build.env.TORCH_CUDA_ARCH_LIST:                    None
build.env.XFORMERS_BUILD_TYPE:                     None
build.env.XFORMERS_ENABLE_DEBUG_ASSERTIONS:        None
build.env.NVCC_FLAGS:                              None
build.env.XFORMERS_PACKAGE_FROM:                   None
source.privacy:                                    open source
'''

def get_python_version_and_parse():
    # Get the current python version
    version = subprocess.run(['python', '-V'], stdout=subprocess.PIPE).stdout.decode().strip()
    version = version.split(' ')[1]
    major, minor, patch = version.split('.')
    # formatted_version = f'cp{major}{minor[0]}{patch[0]}'
    formatted_version = f'cp{major}{minor[0]}'
    return formatted_version


def get_xformers_info():
    get_xformers_info_args = base_args[:]
    get_xformers_info_args += [
        f'cd {webUIDir}','&&',
        'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m xformers.info'
    ]
    get_xformers_info_cmd = (" ").join(get_xformers_info_args)
    print(get_xformers_info_cmd)
    output = !$get_xformers_info_cmd
    output = '\n'.join(output)
    info = {}
    for line in output.strip().split('\n'):
        if ':' in line:
            key, value = line.split(':', 1)
            info[key.strip()] = value.strip()
    return info

def check_xformers_info(validate_rules):
    info = get_xformers_info()
    print((" ").join(info))
    for key, value in validate_rules.items():
        if info.get(key) != value:
            if isinstance(value, bool):
                if str(value) != info.get(key):
                    return False
            else:
                return False
    return True

def find_xformers_whl_and_install(root):
    py_short_ver = get_python_version_and_parse()
    whl_name = f'xformers*{py_short_ver}*.whl'
    whl_path_list = !find $root -name $whl_name
    whl_path_list = list(filter(lambda x: x.endswith('.whl'), whl_path_list))
    print(whl_path_list)
    if whl_path_list:
        
        whl_path = whl_path_list[0]
        # subprocess.run(['pip', 'install', whl_path], stdout=subprocess.PIPE)
        !pip install $whl_path --user 
        return whl_path
    else:
        return None

validate_rules = {
    'is_triton_available': True,
    'pytorch.cuda': 'available',
}

# 首先判断本机是否已经安装了可以运行的Xformers

if check_xformers_info(validate_rules):
    print('Xformers可以正常运行，无需另外安装或编译')
else:
    # 寻找Xformers预编译wheel，并安装返回列表中的第一个wheel
    for i in range(0, 5):  # 在input中进行搜索
        whl_path = find_xformers_whl_and_install(f'/input{i}')
        if whl_path:
            break
        else:
            continue
    

In [ ]:
# # 安装完再次验证Xformers是否能正常运作：
# if check_xformers_info(validate_rules):
#     print(f'安装后可以正常运行，无需重新编译')
# else:
#     print('需要在本机进行Xformers的编译')
#     # (Optional) Makes the build much faster
#     !pip install ninja
#     # Set TORCH_CUDA_ARCH_LIST if running and building on different GPU types
#     # !pip install -v -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers

#     xformers_repo_dir = os.path.join(root_dir,'xformers')

#     if not os.path.exists(xformers_repo_dir):
#         !cd $root_dir &&\
#         git clone https://github.com/facebookresearch/xformers.git

#     # 记录一次编译所需耗时为 6m 17.51s
#     !cd $root_dir &&\
#     cd xformers &&\
#     git show-ref main &&\
#     git symbolic-ref -q HEAD &&\
#     git submodule update --init --recursive -q &&\
#     python setup.py egg_info &&\
#     cd xformers.egg-info &&\
#     pip install -r requires.txt &&\
#     cd .. &&\
#     git rev-parse HEAD &&\
#     python setup.py bdist_wheel

#     # 安装完成后，找到对应的whl，使用pip进行安装
#     find_xformers_whl_and_install('/')

## <i class="fa fa-download"></i> 更新本体与环境

更新Stable-diffusion-WebUI本体与环境命令

## 获取webUIDir

In [ ]:
global webUIDir
try:
    webUIDir
    if len(webUIDir)==0:
        webUIDir = getWebUIDir()
except:
    webUIDir = getWebUIDir()
print(webUIDir)

In [ ]:
from IPython.display import display,clear_output
clear_output()
import ipywidgets as widgets
from func.env import setProxy,getWebUIDir

debugging=False

alertBtn = widgets.Button(
    value=False,
    description='',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='',
    icon='' # (FontAwesome names without the `fa-` prefix)
)


display(alertBtn)
    
if os.path.exists(webUIDir):
    
    alertBtn.description='正在更新...'
    alertBtn.button_style='warning'
    alertBtn.icon='upload'
    
    update_sd_webui_args = base_args[:]
    update_sd_webui_args += [
        f'cd {webUIDir}','&&',
        'git pull','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m pip install -r requirements_versions.txt --user'
    ]
    update_sd_webui_cmd = (" ").join(update_sd_webui_args)
    print(update_sd_webui_cmd)
    !$update_sd_webui_cmd

    if debugging==False:
        # clear_output()
        display(alertBtn)
    
    alertBtn.description='更新完成!'
    alertBtn.button_style='success'
    alertBtn.icon='check'

else:
    alertBtn.description='安装SDWebUI...'
    alertBtn.button_style='info'
    alertBtn.icon='download'
    sd_webui_download_args = base_args[:]
    sd_webui_download_args += [
        f'cd {root_dir}','&&',
        'git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui',
        '&&',
        'cd stable-diffusion-webui','&&',
        # 'git checkout d8f8bcb','&&'
        'python -V',
        '&&',
        # 'conda config --append channels conda-forge','&&',
        # 'conda install --yes --file requirements.txt','&&',
        'python -m pip install -r requirements_versions.txt --user'
    ]
    sd_webui_download_cmd = (" ").join(sd_webui_download_args)
    print(sd_webui_download_cmd)
    !$sd_webui_download_cmd
    
#     r = !source activate &&\
#     conda activate py3.10.6 &&\
#     $proxy &&\
#     cd $webUIDir &&\
#     git pull &&\
#     python -V &&\
#     python -m pip install -r requirements.txt

#     display(r)
    
#     if 'Already up to date.' in r:
#         alertBtn.description='更新完成!'
#         alertBtn.button_style='success'
#         alertBtn.icon='check'
#     else:
#         alertBtn.description='更新失败!'
#         alertBtn.button_style='danger'
#         alertBtn.icon='exclamation'


# clear_output(wait=True)

## 清理多余依赖并安装特定的依赖

In [ ]:
# pip_install_args = base_args[:]

# pip_install_args += [
#     'python -m pip uninstall opencv-python opencv-python-headless opencv-contrib-python -y','&&',
#     'pip install opencv-python==4.5.5.64','&&',
#     'pip install triton==2.0.0.dev20221120',
# ]

# pip_install_cmd = (" ").join(pip_install_args)

# !$pip_install_cmd

## <i class="fa fa-language"></i> 安装中文语言包

In [ ]:
%cd $repo_root_dir
%run ./nbfunc/hub.ipynb

extURL='https://github.com/dtlnor/stable-diffusion-webui-localization-zh_CN'
shallow_clone_ext_repo(extURL)

## <i class="fa fa-images"></i> 安装图库浏览器

In [ ]:
extURL='https://github.com/AlUlkesh/stable-diffusion-webui-images-browser'
shallow_clone_ext_repo(extURL)

## 安装sd_save_intermediate_images插件(保存预测过程的中间图像)

In [ ]:
extURL='https://github.com/AlUlkesh/sd_save_intermediate_images'
shallow_clone_ext_repo(extURL)

## 安装openpose-editor

In [ ]:
extURL='https://github.com/fkunn1326/openpose-editor'
shallow_clone_ext_repo(extURL)

## 安装CLIP Interrogator extension

In [ ]:
extURL='https://github.com/pharmapsychotic/clip-interrogator-ext'
shallow_clone_ext_repo(extURL)

## 安装Additonal-Networks

In [ ]:
extURL='https://github.com/kohya-ss/sd-webui-additional-networks'
shallow_clone_ext_repo(extURL)

## 安装sd_web_ui_preset_utils并载入预设

In [ ]:
extURL='https://github.com/Gerschel/sd_web_ui_preset_utils'
shallow_clone_ext_repo(extURL)

import os

def create_symbolic_link(root_dir):
    # 定义原始文件和目标文件的路径
    original_path = os.path.join(root_dir, 'stable-diffusion-webui', 'extensions', 'sd_web_ui_preset_utils', 'presets.json')
    target_path = os.path.join(root_dir, 'OneClick-stable-diffusion', 'configs', 'sd_web_ui_preset_utils', 'presets.json')
    
    # 删除原始文件（如果存在）
    if os.path.exists(original_path):
        os.remove(original_path)
    
    # 创建符号链接
    os.symlink(target_path, original_path)
    
    print(f"符号链接已成功创建：{original_path} -> {target_path}")

create_symbolic_link(root_dir)

## 安装sd-webui-ar

In [ ]:
extURL='https://github.com/alemelis/sd-webui-ar'
shallow_clone_ext_repo(extURL)

## 安装 Aspect Ratio Helper
📐 Aspect Ratio Helper 是一个简单的扩展，可以在改变尺寸时轻松地保持宽高比。该扩展可以通过 AUTOMATIC1111 webui 的扩展选项卡进行安装。它有以下几个主要功能：
按最大宽度或高度缩放
按百分比缩放
可以通过设置进行自定义，如是否默认展开等。

In [ ]:
extURL='https://github.com/thomasasfk/sd-webui-aspect-ratio-helper'
shallow_clone_ext_repo(extURL)

## 安装depthmap2mask

In [ ]:
extURL='https://github.com/Extraltodeus/depthmap2mask'
shallow_clone_ext_repo(extURL)

## <i class="fa fa-running"></i> 安装ControlNet插件

In [ ]:
extURL='https://github.com/Mikubill/sd-webui-controlnet'
shallow_clone_ext_repo(extURL)

# 在使用segmentation这个preprocessor的时候需要，会遇到报错，通过以下方法进行修复：
# https://github.com/lllyasviel/ControlNet/issues/61#issuecomment-1434005703
# https://github.com/Mikubill/sd-webui-controlnet/issues/70

# 这种方法试过，但不生效

# uninstall_mmcv_args = base_args[:]
# uninstall_mmcv_args += [
#     'pip uninstall mmcv -y'
# ]
# uninstall_mmcv_cmd = (" ").join(uninstall_mmcv_args)
# !$uninstall_mmcv_cmd

#以下这种方法会更改了pythonpath的环境，影响其他库的调用，如SwinIR
#事实上这个说法并不对，是进行了以下环境配置改变之后，使用SwinIR_4x的名称由SwinIR 4x变成SwinIR_4x，多了下划线

import os
from func.env import getExtDir

extDir = getExtDir()

def find_and_set_uniformer_path(dir_name):
    uniformer_path = None
    # 遍历指定目录及其子目录下的所有文件和目录
    for root, dirs, files in os.walk(dir_name):
        # 如果uniformer是一个目录，检查是否在dirs中
        if 'uniformer' in dirs:
            # 获取uniformer所在的完整路径，并打印出来
            uniformer_path = os.path.join(root, 'uniformer')
            print(uniformer_path)
            # 退出循环，不再继续遍历其他目录和文件
            break

        # 如果uniformer是一个文件，检查是否在files中
        if 'uniformer' in files:
            # 获取uniformer所在的完整路径，并打印出来
            uniformer_path = os.path.join(root, 'uniformer')
            print(uniformer_path)
            # 退出循环，不再继续遍历其他目录和文件
            break

    if uniformer_path is not None:
        # 获取原有的PYTHONPATH环境变量的值，并将新的路径追加在后面
        python_path = os.environ.get('PYTHONPATH', '') + os.pathsep + uniformer_path
        # 设置PYTHONPATH环境变量为最新的值
        os.environ['PYTHONPATH'] = python_path
        # 验证环境变量已经设置成功
        verify_uniformer_path_set(uniformer_path)
    else:
        print("在目录{}中未找到uniformer".format(dir_name))

def verify_uniformer_path_set(uniformer_path):
    # 获取当前PYTHONPATH环境变量的值，并检查uniformer所在的路径是否在其中
    python_path = os.environ.get('PYTHONPATH', '')
    print(f'python_path为：{python_path}')
    if uniformer_path in python_path.split(os.pathsep):
        print("PYTHONPATH环境变量设置成功")
    else:
        print("PYTHONPATH环境变量设置失败")

# 调用函数，并传入要搜索的目录名
find_and_set_uniformer_path(extDir)


# 硬盘读写测试

In [ ]:
import os
import time

def test_disk_speed(path='/root/autodl-fs', size=2):
    """测试硬盘读写速度"""
    # 获取当前工作目录并创建测试文件目录
    directory = os.path.join(os.getcwd(), 'test')
    if not os.path.exists(directory):
        os.makedirs(directory)

    # 创建指定大小的虚拟文件
    file_path = os.path.join(directory, 'testfile')
    file_size = int(size * 1024 * 1024)  # MB
    with open(file_path, 'wb') as f:
        f.seek(file_size - 1)
        f.write(b'\0')

    # 测试写入速度
    write_start = time.time()
    with open(os.path.join(path, 'testfile'), 'wb') as f:
        with open(file_path, 'rb') as g:
            f.write(g.read())
    write_end = time.time()
    write_time = write_end - write_start
    print(f'写入速度: {file_size / write_time / 1024 / 1024:.2f} MB/s')

    # 测试读取速度
    read_start = time.time()
    with open(os.path.join(path, 'testfile'), 'rb') as f:
        with open(os.path.join(directory, 'testfile_read'), 'wb') as g:
            g.write(f.read())
    read_end = time.time()
    read_time = read_end - read_start
    print(f'读取速度: {file_size / read_time / 1024 / 1024:.2f} MB/s')

    # 删除虚拟文件
    os.remove(file_path)
    os.remove(os.path.join(directory, 'testfile_read'))
    
# test_disk_speed('/root/autodl-fs', 100)  # 测试 100 MB 大小的文件并写入到指定路径 

## 交代不同环境的挂载方式

In [ ]:
if env_name == 'AutoDL':
    copy_or_mount = True
elif env_name == 'OpenBayes':
    copy_or_mount = True

## 挂载.ckpt或.safetensors到SD WebUI模型文件夹

In [ ]:
import yaml

def read_yaml_file(file_path):
    with open(file_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    return yaml_data

# 调用函数并传入YAML文件路径
models_yaml_path = os.path.join(repo_root_dir,'./configs/models_list/models.yaml')
models_list = read_yaml_file(models_yaml_path)

In [ ]:
# Stable-diffusion 模型
sd_models = models_list['sd_models']

# 遍历模型列表并创建符号链接
for model in sd_models:
    check_model_and_make_symbollink(model, './models/Stable-diffusion', copy_or_mount)

# VAE模型
vae_models = models_list['vae_models']

# 遍历模型列表并创建符号链接
for model in vae_models:
    check_model_and_make_symbollink(model, './models/VAE', copy_or_mount)

# 挂载BLIP模型

In [ ]:
import os

torch_hub_checkpoints_dir = os.path.abspath(os.path.join(root_dir,'.torch/hub/checkpoints/'))
print(torch_hub_checkpoints_dir)
# 该模型的下载地址为：https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth
check_model_and_make_symbollink('model_large_caption.pth',torch_hub_checkpoints_dir,copy_or_mount)

# Clip-Interrogator 模型
vit_models = [
    'ViT-L-14_openai_artists.safetensors',
    'ViT-L-14_openai_flavors.safetensors',
    'ViT-L-14_openai_mediums.safetensors',
    'ViT-L-14_openai_movements.safetensors',
    'ViT-L-14_openai_negative.safetensors',
    'ViT-L-14_openai_trendings.safetensors'
]

# 遍历模型列表并创建符号链接
for model in vit_models:
    check_model_and_make_symbollink(model, './models/clip-interrogator', copy_or_mount)

# 挂载 body pose 模型给 Openpose Editor

In [ ]:
check_model_and_make_symbollink('body_pose_model.pth','./models',copy_or_mount)

## 挂载TI·Embedding

In [ ]:
# 嵌入模型
embeddings = models_list['embeddings']

# 遍历模型列表并创建符号链接
for model in embeddings:
    check_model_and_make_symbollink(model, './embeddings', copy_or_mount)

# 挂载LoRA模型

In [ ]:
import os

def create_model_with_thumbnail(model_name, models_dir, use_symlink=True):
    #先复制LoRA模型
    input_dir = check_model_and_make_symbollink(model_name, models_dir, use_symlink)
    
    if input_dir:
        file_name, ext = os.path.splitext(model_name)
        thumbnail_name = file_name + '.png'
        thumbnail_path = os.path.join(input_dir, thumbnail_name)

        thumbnail_exts = ['png', 'jpg', 'jpeg', 'bmp', 'gif']
        for ext in thumbnail_exts:
            thumbnail_name = file_name + '.' + ext
            thumbnail_path = os.path.join(input_dir, thumbnail_name)
            if os.path.exists(thumbnail_path):
                print(f'找到缩略图：{thumbnail_name}')
                # 再复制缩略图
                check_model_and_make_symbollink(thumbnail_name, models_dir, use_symlink)
                return
        print('未找到缩略图')

SD_orig_Lora_dir = './models/Lora'
Additional_Networks_Lora_dir = './extensions/sd-webui-additional-networks/models/lora'

create_symlink(
    source_path = os.path.abspath(os.path.join(webUIDir,SD_orig_Lora_dir)),
    symlink_path = os.path.abspath(os.path.join(webUIDir,Additional_Networks_Lora_dir)),
)

for i in [SD_orig_Lora_dir]:
    # 模型
    LoRA_models = models_list['LoRA_models']

    # 遍历模型列表并创建模型
    for model in LoRA_models:
        create_model_with_thumbnail(model, i, copy_or_mount)

## 部署其他必要模型

### 图生文模型

In [ ]:
# BLIP
check_model_and_make_symbollink('model_base_caption_capfilt_large.pth','./models/BLIP',copy_or_mount)
check_model_and_make_symbollink('ViT-L-14.pt','/root/.cache/clip',copy_or_mount)
check_model_and_make_symbollink('ViT-B-16.pt','/root/.cache/clip',copy_or_mount)
# Deepbooru模型
check_model_and_make_symbollink('model-resnet_custom_v3.pt','./models/torch_deepdanbooru',copy_or_mount)

### 人脸修复模型

In [ ]:
# GFPGAN
check_model_and_make_symbollink('GFPGANv1.4.pth','./models/GFPGAN',copy_or_mount)
check_model_and_make_symbollink('detection_Resnet50_Final.pth','./models/GFPGAN',copy_or_mount)
check_model_and_make_symbollink('parsing_parsenet.pth','./models/GFPGAN',copy_or_mount)
# Codeformer
check_model_and_make_symbollink('codeformer-v0.1.0.pth','./models/Codeformer',copy_or_mount)
check_model_and_make_symbollink('detection_Resnet50_Final.pth','./repositories/CodeFormer/weights/facelib/',copy_or_mount)

### Upscale模型

In [ ]:
check_model_and_make_symbollink('SwinIR_4x.pth','./models/SwinIR',copy_or_mount)
check_model_and_make_symbollink('RealESRGAN_x4plus.pth','./models/RealESRGAN',copy_or_mount)
check_model_and_make_symbollink('BSRGAN.pth','./models/ESRGAN',copy_or_mount)
check_model_and_make_symbollink('4x_foolhardy_Remacri.pth','./models/ESRGAN',copy_or_mount)
check_model_and_make_symbollink('ESRGAN_4x.pth','./models/ESRGAN',copy_or_mount)

## 部署ControlNet模型
解压ControlNet模型并在controlnet拓展的模型目录下创建对应模型的符号链接

### ControlNet的preprocessors模型

In [ ]:
# 模型
models = [
    ('dpt_hybrid-midas-501f0c75.pt', 'midas'),  # midas模型
    ('res101.pth', 'leres'),  # res101.pth模型
    ('network-bsds500.pth', 'hed'),  # hed模型
    ('mlsd_large_512_fp32.pth', 'mlsd'),  # mlsd模型
    ('body_pose_model.pth', 'openpose'),  # body_pose_model模型
    ('hand_pose_model.pth', 'openpose'),  # hand_pose_model模型
    ('upernet_global_small.pth', 'uniformer')  # Segmentation的preprocessor模型
]

# 遍历模型列表并创建符号链接
for model, directory in models:
    check_model_and_make_symbollink(model, f'./extensions/sd-webui-controlnet/annotator/{directory}', copy_or_mount)

### ControlNet的模型

In [ ]:
# 模型名称
controlnet_models = [
    'control_canny-fp16.safetensors',
    'control_depth-fp16.safetensors',
    'control_hed-fp16.safetensors',
    'control_mlsd-fp16.safetensors',
    'control_normal-fp16.safetensors',
    'control_openpose-fp16.safetensors',
    'control_scribble-fp16.safetensors',
    'control_seg-fp16.safetensors'
]

# 遍历模型名称列表并创建符号链接
for model_name in controlnet_models:
    check_model_and_make_symbollink(model_name, './extensions/sd-webui-controlnet/models', copy_or_mount)

### T2I模型

In [ ]:
# 模型名称
t2i_models = [ 
    ('t2iadapter_canny_sd14v1.pth', 'sketch_adapter_v14.yaml'), 
    ('t2iadapter_sketch_sd14v1.pth', 'sketch_adapter_v14.yaml'),
    ('t2iadapter_seg_sd14v1.pth', 'image_adapter_v14.yaml'),
    ('t2iadapter_keypose_sd14v1.pth', 'image_adapter_v14.yaml'),  
    ('t2iadapter_openpose_sd14v1.pth', 'image_adapter_v14.yaml'), 
    ('t2iadapter_color_sd14v1.pth', 't2iadapter_color_sd14v1.yaml'), 
    ('t2iadapter_style_sd14v1.pth', 't2iadapter_style_sd14v1.yaml')
]

def copy_and_rename_yaml(ref_yaml, model_name, target_dir):
    """
    复制并重命名给定的配置文件。
    将源文件复制到目标目录，重命名为与模型名称相同的名称（不包括扩展名）。
    Args:
        ref_yaml (str): 需要复制的配置文件的文件名。
        model_name (str): 目标模型的名称。
        target_dir (str): 目标目录的路径。
    """
    # 从 ref_yaml 中提取文件名和扩展名
    ref_name, ref_ext = os.path.splitext(ref_yaml)

    # 构建目标文件名
    target_name = os.path.splitext(model_name)[0] + ref_ext
    target_path = os.path.join(target_dir, target_name)
    
    # 检查目标路径是否存在相应的文件
    if os.path.exists(target_path):
        print(f"{target_path} 已经存在,跳过复制")
        return
    
    yaml_path = os.path.join(target_dir, ref_yaml)

    # 复制源文件并将其重命名为目标文件名
    shutil.copyfile(yaml_path, target_path)

# 遍历模型名称列表并创建符号链接
target_dir = os.path.join(webUIDir, './extensions/sd-webui-controlnet/models')

for model_name, ref_yaml in t2i_models:
    check_model_and_make_symbollink(model_name, target_dir, copy_or_mount)
    copy_and_rename_yaml(ref_yaml, model_name, target_dir)

## 下载v2模型对应的YAML配置文件

In [ ]:
global webUIDir
try:
    webUIDir
    if len(webUIDir)==0:
        webUIDir = getWebUIDir()
except:
    webUIDir = getWebUIDir()
print(webUIDir)
    
sd_models_dir = os.path.realpath(os.path.join(webUIDir,'./models/Stable-diffusion'))
print(sd_models_dir)
!$proxy &&\
cd $sd_models_dir &&\
wget -O v2-1_768-nonema-pruned.yaml https://raw.githubusercontent.com/Stability-AI/stablediffusion/main/configs/stable-diffusion/v2-inference-v.yaml

## 复制repos

In [ ]:
import os

global webUIDir
try:
    webUIDir
    if len(webUIDir)==0:
        webUIDir = getWebUIDir()
except:
    webUIDir = getWebUIDir()
print(webUIDir)

def search_dir_structure(root_dir, structure):
    """
    Searches for a specific directory structure in a given root directory.
    root_dir: str, the root directory to search in
    structure: dict, the directory structure to search for
    """
    dirpaths=[]
    for dirpath, dirnames, filenames in os.walk(root_dir):
        match = True
        for subdir in structure:
            if subdir not in dirnames:
                match = False
                break
            # Check if the subdirectory has the same structure as defined in the structure
            subdir_path = os.path.join(dirpath, subdir)
            match = match and search_dir_structure(subdir_path, structure[subdir])
        if match:
             dirpaths.append(dirpath)    
    return dirpaths
        
def copy_repos_for_sd_webui():
    structure = {
        "BLIP": {
            "data": {},
            "models": {}
        },
        "CodeFormer": {

        },
        "k-diffusion": {},
        "stable-diffusion-stability-ai": {},
        "taming-transformers": {}
    }

    dir_list = search_dir_structure("/", structure)

    input_repos_dir = None

    for i in dir_list:
        if 'input' in i:
            input_repos_dir = i
            break

    repos_dir = os.path.realpath(os.path.join(webUIDir,'./repositories'))

    print(input_repos_dir)
    print(os.path.dirname(repos_dir))
    repos_in_sd_list = search_dir_structure(os.path.dirname(repos_dir), structure)

    repos_in_sd_dir_toTest = None
    if len(repos_in_sd_list)>0:
        for i in dir_list:
            if 'stable-diffusion-webui' in i:
                repos_in_sd_dir_toTest = i
                break

    if input_repos_dir:
        if os.path.exists(repos_dir) and search_dir_structure(os.path.dirname(repos_dir), structure):
            print(f"{repos_dir} 已经存在并拥有正确的Repos")
        else:
            if not os.path.exists(repos_dir):
                !mkdir $repos_dir
            # !cp -r $input_repos_dir/* $repos_dir
            if os.path.exists(repos_dir):
                !rm -rf $repos_dir
            !cp -r $input_repos_dir $repos_dir # 创建软链会导致只能可读，不能覆写
            print('已完成复制！')
    else:
        print('找不到对应的数据集')
        
        
if env_name == 'AutoDL':
    print('AutoDL环境无需运行这个cell')
    pass
elif env_name == 'OpenBayes':
    copy_repos_for_sd_webui()

# 生成Config.json

## 定义要写入的JSON对象（太长了，建议折叠）

In [ ]:
# 定义要写入的JSON对象
config = {
    "samples_save": True,
    "samples_format": "png",
    "samples_filename_pattern": "",
    "save_images_add_number": True,
    "grid_save": True,
    "grid_format": "png",
    "grid_extended_filename": False,
    "grid_only_if_multiple": True,
    "grid_prevent_empty_spots": False,
    "n_rows": -1,
    "enable_pnginfo": True,
    "save_txt": False,
    "save_images_before_face_restoration": False,
    "save_images_before_highres_fix": False,
    "save_images_before_color_correction": False,
    "jpeg_quality": 80,
    "export_for_4chan": True,
    "img_downscale_threshold": 4.0,
    "target_side_length": 4000,
    "use_original_name_batch": True,
    "use_upscaler_name_as_suffix": False,
    "save_selected_only": True,
    "do_not_add_watermark": False,
    "temp_dir": "",
    "clean_temp_dir_at_start": False,
    "outdir_samples": "",
    "outdir_txt2img_samples": "outputs/txt2img-images",
    "outdir_img2img_samples": "outputs/img2img-images",
    "outdir_extras_samples": "outputs/extras-images",
    "outdir_grids": "",
    "outdir_txt2img_grids": "outputs/txt2img-grids",
    "outdir_img2img_grids": "outputs/img2img-grids",
    "outdir_save": "log/images",
    "save_to_dirs": True,
    "grid_save_to_dirs": True,
    "use_save_to_dirs_for_ui": False,
    "directories_filename_pattern": "[date]",
    "directories_max_prompt_words": 8,
    "ESRGAN_tile": 192,
    "ESRGAN_tile_overlap": 8,
    "realesrgan_enabled_models": [
        "R-ESRGAN 4x+",
        "R-ESRGAN 4x+ Anime6B"
    ],
    "upscaler_for_img2img": None,
    "face_restoration_model": "CodeFormer",
    "code_former_weight": 0.5,
    "face_restoration_unload": False,
    "show_warnings": False,
    "memmon_poll_rate": 8,
    "samples_log_stdout": False,
    "multiple_tqdm": True,
    "print_hypernet_extra": False,
    "unload_models_when_training": False,
    "pin_memory": False,
    "save_optimizer_state": False,
    "save_training_settings_to_txt": True,
    "dataset_filename_word_regex": "",
    "dataset_filename_join_string": " ",
    "training_image_repeats_per_epoch": 1,
    "training_write_csv_every": 500,
    "training_xattention_optimizations": False,
    "training_enable_tensorboard": False,
    "training_tensorboard_save_images": False,
    "training_tensorboard_flush_every": 120,
    "sd_model_checkpoint": "chilloutmix_NiPrunedFp16.safetensors [1db9eb3bc3]",
    "sd_checkpoint_cache": 0,
    "sd_vae_checkpoint_cache": 0,
    "sd_vae": "Automatic",
    "sd_vae_as_default": True,
    "inpainting_mask_weight": 1.0,
    "initial_noise_multiplier": 1.0,
    "img2img_color_correction": False,
    "img2img_fix_steps": False,
    "img2img_background_color": "#ffffff",
    "enable_quantization": False,
    "enable_emphasis": True,
    "enable_batch_seeds": True,
    "comma_padding_backtrack": 20,
    "CLIP_stop_at_last_layers": 1,
    "upcast_attn": False,
    "use_old_emphasis_implementation": False,
    "use_old_karras_scheduler_sigmas": False,
    "no_dpmpp_sde_batch_determinism": False,
    "use_old_hires_fix_width_height": False,
    "interrogate_keep_models_in_memory": False,
    "interrogate_return_ranks": False,
    "interrogate_clip_num_beams": 1,
    "interrogate_clip_min_length": 24,
    "interrogate_clip_max_length": 48,
    "interrogate_clip_dict_limit": 1500,
    "interrogate_clip_skip_categories": [],
    "interrogate_deepbooru_score_threshold": 0.5,
    "deepbooru_sort_alpha": True,
    "deepbooru_use_spaces": False,
    "deepbooru_escape": True,
    "deepbooru_filter_tags": "",
    "extra_networks_default_view": "cards",
    "extra_networks_default_multiplier": 1.0,
    "sd_hypernetwork": "None",
    "return_grid": True,
    "do_not_show_images": False,
    "add_model_hash_to_info": True,
    "add_model_name_to_info": True,
    "disable_weights_auto_swap": True,
    "send_seed": True,
    "send_size": True,
    "font": "",
    "js_modal_lightbox": True,
    "js_modal_lightbox_initially_zoomed": True,
    "show_progress_in_title": True,
    "samplers_in_dropdown": True,
    "dimensions_and_batch_together": True,
    "keyedit_precision_attention": 0.1,
    "keyedit_precision_extra": 0.05,
    "quicksettings": "sd_model_checkpoint",
    "ui_reorder": "inpaint, sampler, checkboxes, hires_fix, dimensions, cfg, seed, batch, override_settings, scripts",
    "ui_extra_networks_tab_reorder": "",
    "localization": "None",
    "show_progressbar": True,
    "live_previews_enable": True,
    "show_progress_grid": True,
    "show_progress_every_n_steps": 10,
    "show_progress_type": "Approx NN",
    "live_preview_content": "Prompt",
    "live_preview_refresh_period": 1000,
    "hide_samplers": [],
    "eta_ddim": 0.0,
    "eta_ancestral": 1.0,
    "ddim_discretize": "uniform",
    "s_churn": 0.0,
    "s_tmin": 0.0,
    "s_noise": 1.0,
    "eta_noise_seed_delta": 0,
    "always_discard_next_to_last_sigma": False,
    "postprocessing_enable_in_main_ui": [],
    "postprocessing_operation_order": [],
    "upscaling_max_images_in_cache": 5,
    "disabled_extensions": [],
    "sd_checkpoint_hash": "1db9eb3bc30c9ab238a602b321953275db081e3bd4702d12b53bccefceca5537",
    "ldsr_steps": 100,
    "ldsr_cached": False,
    "SWIN_tile": 192,
    "SWIN_tile_overlap": 8,
    "sd_lora": "None",
    "lora_apply_to_outputs": False,
    "control_net_model_config": "/output/stable-diffusion-webui/extensions/sd-webui-controlnet/models/cldm_v15.yaml",
    "control_net_model_adapter_config": "/output/stable-diffusion-webui/extensions/sd-webui-controlnet/models/sketch_adapter_v14.yaml",
    "control_net_detectedmap_dir": "/output/stable-diffusion-webui/extensions/sd-webui-controlnet/detected_maps",
    "control_net_models_path": "",
    "control_net_max_models_num": 1,
    "control_net_model_cache_size": 2,
    "control_net_control_transfer": False,
    "control_net_no_detectmap": False,
    "control_net_detectmap_autosaving": False,
    "control_net_only_midctrl_hires": True,
    "control_net_allow_script_control": False,
    "control_net_skip_img2img_processing": False,
    "control_net_monocular_depth_optim": False,
    "control_net_only_mid_control": False,
    "control_net_cfg_based_guidance": False,
    "image_browser_with_subdirs": True,
    "image_browser_preload": False,
    "image_browser_copy_image": False,
    "image_browser_delete_message": True,
    "image_browser_txt_files": True,
    "image_browser_logger_warning": False,
    "image_browser_logger_debug": False,
    "image_browser_delete_recycle": False,
    "image_browser_scan_exif": True,
    "image_browser_mod_shift": False,
    "image_browser_mod_ctrl_shift": False,
    "image_browser_page_columns": 6.0,
    "image_browser_page_rows": 6.0,
    "image_browser_pages_perload": 20.0
}

## 生成Config.json的执行操作

In [ ]:
import json
import os
from func.env import getWebUIDir

global webUIDir
try:
    webUIDir
    if len(webUIDir)==0:
        webUIDir = getWebUIDir()
except:
    webUIDir = getWebUIDir()
print(webUIDir)

# 定义文件路径
file_path = os.path.join(webUIDir, "config.json")

# 将JSON对象写入文件
with open(file_path, "w") as f:
    json.dump(config, f)

## 对SD-WebUI的config.json进行参数配置

In [ ]:
import os
import json
from shutil import copyfile
from func.env import getWebUIDir
from datetime import datetime

def modify_config(modify_func, backup_folder="config_backups"):
    global webUIDir
    try:
        webUIDir
        if len(webUIDir)==0:
            webUIDir = getWebUIDir()
    except:
        webUIDir = getWebUIDir()
    print(webUIDir)

    config_file_path = os.path.join(webUIDir, "config.json")
    if os.path.exists(config_file_path):
        # 如果文件存在，就执行后续的操作
        backup_folder_path = os.path.join(webUIDir, backup_folder)
        if not os.path.exists(backup_folder_path):
            os.makedirs(backup_folder_path)
        backup_file_path = os.path.join(backup_folder_path, f"config_backup_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json")

        # 备份文件
        copyfile(config_file_path, backup_file_path)

        with open(config_file_path, "r") as f:
            config = json.load(f)

        config = modify_func(config)

        with open(config_file_path, "w") as f:
            json.dump(config, f)

        print(f"成功修改并备份了 {config_file_path} 文件至 {backup_file_path}！")

    else:
        # 如果文件不存在，就不进行修改
        print("config.json文件不存在，无法修改！")

# 使用示例，修改 quicksettings 的值
def modify(config):
    # 顶部快捷方式呈现：模型选择、VAE选择、内绘遮罩权重、Clip Skip系数调节
    config['quicksettings'] = "sd_model_checkpoint, sd_vae, inpainting_mask_weight, CLIP_stop_at_last_layers"
    # 图生图的超分辨率算法
    config['upscaler_for_img2img'] = "R-ESRGAN 4x+"
    # Extra Networks的默认视图
    config['extra_networks_default_view'] = "thumbs"
    # Extra Networks的默认系数
    config['extra_networks_default_multiplier'] = 0.6
    
    # ControlNet多开的数量
    config['control_net_max_models_num'] = 8
    # ControlNet预处理器图片自动保存
    config['control_net_detectmap_autosaving'] = True
    # 允许其他脚本控制ControlNet拓展
    config['control_net_allow_script_control'] = True
    # 启用基于CFG的引导(CFG-Based guidance)（配合无提示词(prompt)模式使用时请删除所有提示词，推荐设置"采样次数：50，CFG：3~5"，ControlNet将会自动识别输入的模式图）
    config['control_net_cfg_based_guidance'] = True
    
    # CLIP skip 系数 2
    config['CLIP_stop_at_last_layers'] = 2
    # 启动网页时对图库浏览器进行预加载
    config['image_browser_preload'] = True
    # 设置语言为中文
    config['localization'] = 'zh_CN'
    # 设置预览步数，默认为10
    config['show_progress_every_n_steps'] = 10

    return config

modify_config(modify, backup_folder="config_backups")

# <i class="fa fa-rocket"></i> 启动Stable-Diffusion-WebUI

In [ ]:
!pip install gradio==3.23 -i https://pypi.org/simple
!pip install clip-interrogator==0.6.0 --prefer-binary -i https://pypi.org/simple
!pip install transformers -i https://pypi.org/simple

In [ ]:
from func.env import setProxy,getWebUIDir,getExtDir,findDir,findFile,getDirSize,styleURL,getArch
import ipywidgets as widgets
from subprocess import getoutput
from func.xformers import checkXformersInstalled
from IPython.display import display,clear_output

extDir=getExtDir()
archCondition = False
canUseXformers = False

cb = getArch()
arch = cb['arch']
gpu = cb['gpu']

whiteList = ['sm80','sm75','sm70','sm86','sm89','sm61']

if arch in whiteList:
    archCondition = True
else:
    archCondition = False
        
if(checkXformersInstalled() and archCondition):
    canUseXformers = True
else:
    canUseXformers = False

canUseXformers = True

global webUIDir
try:
    webUIDir
    if len(webUIDir)==0:
        webUIDir = getWebUIDir()
except:
    webUIDir = getWebUIDir()
print(webUIDir)
    
args = base_args[:]

args += [
    f'cd {webUIDir}','&&',
    # 'pip install "opencv-python-headless<4.3" ','&&',
    'python launch.py',
    '--share',
    '--opt-split-attention', #–opt-split-attention是一个优化选项，它可以让你在有限的显存下生成更大的图片。它的原理是将注意力机制分割成多个头部，减少计算量和内存消耗。如果你不开启这个选项，你可能会遇到显存不足的问题。
    '--deepdanbooru',
    '--administrator',
    '--disable-safe-unpickle',
    '--no-half-vae',
    '--disable-nan-check',
    '--api'
    # '--no-half'
       ]

# try:
#     proxy
# except NameError:
#     print("不存在proxy")
# else:
#     if region=='未知':
#         print("不存在proxy")
#     else:
#         print(f"已经挂载【{region}】proxy")
#         args.insert(0,f"{proxy} && env|grep -i proxy &&")

if(canUseXformers==True):
    print("可以使用Xformers")
    args.append("--xformers")
else:
    print(f"暂不支持【{gpu}】该型号GPU，不使用Xformers")

if env_name == 'AutoDL':
    args.append("--port 6006")
elif env_name == 'OpenBayes':
    args.append("--port 7860")
    
cmd = (" ").join(args)
print(cmd)

!$cmd